In [15]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#Import API key
from config import g_key

In [16]:
#Import weather_data.csv from WeatherPY 

weather_data = pd.read_csv('../WeatherPY/weather_data.csv')
weather_data

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rio Grande,-32.0350,-52.0986,300.15,50,20,4.12,BR,1615558960
1,Hermanus,-34.4187,19.2345,296.48,67,100,0.45,ZA,1615558962
2,Qaanaaq,77.4840,-69.3632,257.07,69,100,0.77,GL,1615558964
3,Avera,33.1940,-82.5271,291.48,82,1,2.06,US,1615558965
4,Rikitea,-23.1203,-134.9692,299.29,76,100,3.78,PF,1615558966
...,...,...,...,...,...,...,...,...,...
566,Ormara,25.2088,64.6357,296.94,81,14,1.63,PK,1615559751
567,Price,39.5994,-110.8107,271.15,74,20,5.14,US,1615559752
568,Samarinda,-0.5000,117.1500,296.55,93,89,0.77,ID,1615559756
569,Teya,60.3778,92.6267,261.90,89,62,4.03,RU,1615559757


# Humidity Heatmap

In [17]:
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.

#Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

#Store humidity into humidity 
humidity = weather_data["Humidity"]

In [18]:
#Add Heatmap layer to map.

#plot heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
#Create new Dataframe fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

max_temp_df = weather_data.loc[(weather_data["Max temp"] <= 265.00)]
#max_temp_df

wind_speed_df = weather_data.loc[(weather_data["Wind Speed"] <= 10)]
#wind_speed_df.head()

cloud_data_df = weather_data.loc[(weather_data["Cloudiness"] == 0)]
#cloud_data_df.head()

In [57]:
#Merge the two dataframes together
specific_city_df = pd.merge(wind_speed_df, cloud_data_df, how="inner", on=None)
specific_cities_df = pd.merge(specific_city_df, max_temp_df, how="inner", on=None)
specific_cities_df

#Remove rows missing data
specific_cities_df = specific_cities_df.dropna()
specific_cities_df


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tarko-Sale,64.9225,77.7850,246.84,88,0,6.42,RU,1615559256
1,Gubkinskiy,64.4340,76.5026,248.47,88,0,6.74,RU,1615559301
2,Upernavik,72.7868,-56.1549,255.52,77,0,0.68,GL,1615559348
3,Arman',59.7000,150.1667,254.57,70,0,3.41,RU,1615559448
4,Konosha,60.9736,40.2570,264.69,83,0,5.27,RU,1615559546
5,Krapivinskiy,54.9992,86.8133,261.15,56,0,3.00,RU,1615559616
6,Līsakovsk,52.5369,62.4936,251.20,85,0,3.99,KZ,1615559648


# Hotel Map

In [58]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.

hotel_df = specific_cities_df.loc[:,["City", "Lat", "Lng", "Max temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Tarko-Sale,64.9225,77.7850,246.84,88,0,6.42,RU,1615559256,
1,Gubkinskiy,64.4340,76.5026,248.47,88,0,6.74,RU,1615559301,
2,Upernavik,72.7868,-56.1549,255.52,77,0,0.68,GL,1615559348,
3,Arman',59.7000,150.1667,254.57,70,0,3.41,RU,1615559448,
4,Konosha,60.9736,40.2570,264.69,83,0,5.27,RU,1615559546,
5,Krapivinskiy,54.9992,86.8133,261.15,56,0,3.00,RU,1615559616,
6,Līsakovsk,52.5369,62.4936,251.20,85,0,3.99,KZ,1615559648,


In [59]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.

#Set up url & lists to hold response info
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
my_params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
     
        lat = row["Lat"]
        lng = row["Lng"]
        city_name = row["City"]
        
        my_params['location'] = f"{lat},{lng}"
        
        print(f"Retrieving Results for Index {index}: {city_name}.")
        response = requests.get(base_url, params=my_params).json()
        #print(response)
        
        results = response["results"]
        
        try:
            print(f"The closest hotel in {city_name} is {results[0]['name']}.")
            hotel_df.loc[index, "Hotel Name"] = results[0]['name']
            
        except (KeyError, IndexError):
            print("Missing field/result...Skipping.")
            
        print("-----------------------")
        print(f'Data Retrieval Complete')
            
        


Retrieving Results for Index 0: Tarko-Sale.
The closest hotel in Tarko-Sale is Yubileynaya.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 1: Gubkinskiy.
The closest hotel in Gubkinskiy is MonarcH.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 2: Upernavik.
The closest hotel in Upernavik is Gina's Guesthouse - Upernavik.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 3: Arman'.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 4: Konosha.
The closest hotel in Konosha is Gostinitsa Miks.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 5: Krapivinskiy.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 6: Līsakovsk.
The closest hotel in Līsakovsk is Gostinitsa Lisakovsk.
-----------------------
Data Retrieval Complete


In [60]:
#Store the first Hotel result into the DataFrame.

hotel_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Tarko-Sale,64.9225,77.7850,246.84,88,0,6.42,RU,1615559256,Yubileynaya
1,Gubkinskiy,64.4340,76.5026,248.47,88,0,6.74,RU,1615559301,MonarcH
2,Upernavik,72.7868,-56.1549,255.52,77,0,0.68,GL,1615559348,Gina's Guesthouse - Upernavik
3,Arman',59.7000,150.1667,254.57,70,0,3.41,RU,1615559448,
4,Konosha,60.9736,40.2570,264.69,83,0,5.27,RU,1615559546,Gostinitsa Miks
5,Krapivinskiy,54.9992,86.8133,261.15,56,0,3.00,RU,1615559616,
6,Līsakovsk,52.5369,62.4936,251.20,85,0,3.99,KZ,1615559648,Gostinitsa Lisakovsk


In [61]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [63]:
# Add marker layer ontop of heat map
markers = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info)

# Display figure
fig = gmaps.figure()
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))